In [65]:
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import uuid
#solved chinese display in matplotlib
from matplotlib import rcParams
rcParams['font.family'] = 'sans-serif'
rcParams["font.family"] = "DFKai-SB"
rcParams['axes.unicode_minus'] = False

from tqdm import tqdm_notebook,tnrange
from tqdm.autonotebook import tqdm
tqdm.pandas()

import regex as re
industry=['金融業', 'IC製造', 'IC設計', '電子零件元件', '電腦系統業', '石油及天然氣', '手機', '面板業', '太陽能',
       '板鋼', '被動元件', '車輛整車', '非鐵金屬', '流通業', '地產', '通訊設備', '遊戲產業', '印刷電路板相關',
       '貴金屬', '機械', '運輸事業', '電子通路', 'IC封裝測試', 'LED', '通訊服務', '消費性電子產品', '化學工業',
       '醫藥產業', '軟體業', '週邊產品', '石化業', '旅館、餐飲', '汽機車零組件', '設備儀器廠商', '條鋼', '造紙業',
       '水泥', '橡膠工業', '傳產其他', '數位相機', '顯示器', '不鏽鋼', '農林漁牧', '服務業', 'Internet相關',
       '營造工程', '休閒娛樂', '紡織中游', '生物科技', '家電', '光碟片', '建材', '成衣', '電力', '運動產業',
       '線材、盤元', '化纖原料', '其他公用事業', '大宗物資', '家居用品', '手機零組件', '食品加工', '電力設備',
       '航天軍工', '分離式元件', '電子其他', '面板零組件', '飲料相關', '電線電纜', '封測服務與材料', '礦石開採',
       '光通訊', '傳播事業', '輔助與彌補用醫材', '時尚產業', '基礎建設營運', '電聲產品', '水資源', '醫療器材通路',
       '合金鋼', '文化創意產業', '車用金屬成型', '穿戴式裝置', '其他醫療器材', '汽車內裝', '車用電子',
       '診斷與監測用醫材', '傳輸介面', '電子化工材料', '電池材料相關', '資產股', '醫療管理服務', '體外診斷用醫材', '煤',
       '射頻前端晶片', '控股公司', '無店舖販售', '手術與治療用醫材', '金屬礦採選', '生物辨識相關']

In [3]:
news_v05=pd.read_pickle("D:4.AutoencoderForArticle.Mid-Industry/news_vip.v05.p")

In [25]:
news_v05.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504261 entries, 0 to 504260
Data columns (total 13 columns):
index                   504261 non-null int64
publishtime             504261 non-null datetime64[ns]
title                   504261 non-null object
body                    504261 non-null object
relateid                504261 non-null object
guid                    504261 non-null object
no_use                  331542 non-null object
tags                    331542 non-null object
title_token             504261 non-null object
body_token              504261 non-null object
LDA_Topic               504261 non-null int8
LDA_Conf                504261 non-null float64
title_and_body_token    504261 non-null object
dtypes: datetime64[ns](1), float64(1), int64(1), int8(1), object(9)
memory usage: 46.6+ MB


In [7]:
newswithindustry_v04=pd.read_pickle("D:4.AutoencoderForArticle.Mid-Industry/news_vip_clean.p")

In [26]:
newswithindustry_v04.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504261 entries, 0 to 504260
Data columns (total 3 columns):
guid                           504261 non-null object
medi-industry                  504261 non-null object
title_token_plus_body_token    504261 non-null object
dtypes: object(3)
memory usage: 11.5+ MB


In [12]:
news_v05["title_and_body_token"]=news_v05["title_token"]+" "+news_v05["body_token"]

In [20]:
title_and_body_token=news_v05["title_and_body_token"]
textcount=title_and_body_token.value_counts(ascending=False)
textcount.reset_index()

,index,title_and_body_token
0,兩岸三地 指數 最新 報價 兩岸三地 指數 最新 指數 漲跌幅 成交 金額 人民幣,13630
1,兩岸三地 指數 報價 兩岸三地 指數 收盤 指數 漲跌幅 成交 金額 人民幣,1750
2,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX 3...,396
3,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX S...,393
4,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX 3...,372
5,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX 3...,283
6,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX 3...,281
7,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX S...,276
8,美股 指數 期貨 最新 報價 指數 漲跌 漲跌幅 資料 時間 時間 DJIA INDEX 3...,241
9,人幣 匯率 中間價 日報 貶值 中國大陸 外匯 交易中心 網站 公佈 銀行間 外匯市場 美元...,238


In [23]:
news_v05_deduplicate=news_v05.drop_duplicates(subset="title_and_body_token")

In [24]:
title_and_body_token=news_v05_deduplicate["title_and_body_token"]
textcount=title_and_body_token.value_counts(ascending=False)
textcount.reset_index()

,index,title_and_body_token
0,亞洲貨幣 升值 態勢 不變 兩岸 水泥 市場 價格 穩定成長 水泥 股長 多可期 由於 美元...,1
1,鐿鈦 Q2 營收 走強 全年 可望 10% 成長 醫材 鐿鈦 4163 參與 櫃買中心 法說...,1
2,秋雨 上海 廠圍 事件 平 落幕 秋雨 印刷 9929 上海 廠因 環保 問題 遭到 當地 ...,1
3,歐股 美 GDP 低於 預期 Saipem 暴跌 三大 指數 收黑 美國 去年 第四季 GD...,1
4,華孚 年報 稅後盈餘 億元 EPS 機殼 廠商 華孚 6235 經 會計師 核閱 去度 財報...,1
5,聰明 消費 3C 賣場 年終 酬賓 刷卡 多賺 一手 又 歲末 年終 消費 旺季 3C 通路...,1
6,震後 成日 本民 眾 支持 廢核 成 擔心 核災 日本 媒體 東京 新聞報導 根據 日本 世...,1
7,油價 供應 增加 NYMEX 原油 週線 跌逾 4% 紐約 商業 交易 NYMEX 原油 期...,1
8,美國 就業報告 遜於 預期 美元 相 對於 歐元 日圓 走 貶 美元 相 對於 歐元 日圓 ...,1
9,高雄港 去年 貨櫃 裝卸量 TEU 年減 全球 主要 港口 去年 貨櫃 裝卸量 陸續 公佈 ...,1


In [27]:
text=news_v05_deduplicate["title_and_body_token"].values

array(['Fund 消息 資金 連 周 回流 收債 後市 看 樂觀 美國 景氣 維持 暢旺 加上 市場 貿易戰 消息 反應 漸有 鈍化 跡象 帶動 高收益債 買氣 增溫 觀察 iShares iBoxx 收債 ETF 不僅 連續 周 獲得 資金 淨流入 上周 為止 累計 淨流入 億美元 逼近 億美元 單月 次 法人 認為 當前 景氣 復甦 趨勢 沒 改變 信用風險 改善 收債 投資 價值 浮現 預期 持續 吸引 收息 興趣 投資人 進場 根據 美銀 美林 公布 資料 顯示 美國 企業 稅前 息前 折舊 攤銷 獲利 EBITDA 年增率 連續 季度 高於 債務 增幅 扭轉 過去 季度 走勢 金融海嘯 以來 首見 代表 企業 財務槓桿 改善 事實 更為 收債 投資人 增添 不少 信心 第一金 全球 高收益債 券 基金經理人 鄭宇君 表示 美國經濟 穩步 復甦 減稅 政策 企業 獲利 帶來 強勁 成長動能 信用 水準 亦 獲得 明顯改善 原先 預期 收債 違約率 將達 截至 月底 為止 過去 一年 平均 違約率 太大意 全年 實際 違約 情形 機會 優於市場預期 此外 鄭宇君 認為 因為 企業 營收 獲利成長 支撐 升息 循環 並未 企業信用 還款 造成 壓力 依據 美國 智庫 NBER 編製 經濟衰退 指標 信用貸款 狀況 透露 目前 銀行業 對於 大型企業 借貸 條件 仍然 寬鬆 過往 衰退 指標 出現 警訊 水準 比較 相距甚遠 顯示 當前 環境 有利於 收債 高收益債 方面 安聯 收益 成長 多重 資產 基金經理人 林素萍則 表示 高收益債 報酬 表現 空間 接近 股票 下檔 風險 又 股票 低 預期 2018 兩年 違約率 仍將 低於 長期 平均 因此 對於 高收益債 後市 偏 多 看待 其中 又 穩健 基本面 支撐 整體 收益率 表現 相對 高的 美國 高收益債 相對 獲 看好 林素萍 進一步 說明 根據 研究 資料 顯示 高收益債 各信評 級別 券種 若 扣除 掉 未來 五年 預期 違約 損失 BB 級 債約 剩下 利差 B 級債 剩下 利差 CCC 級 債則 剩下 利差 考慮 風險 報酬 來看 BB 級債 B 級債 較佳 選擇 周高 收債 ETF 資金 淨流入 情形 億美元 資料 來源 Bloomberg 統計 日為止',
       'Fund 

In [36]:
text.shape

(473966,)

In [37]:
len(set(text))

473966

In [99]:
news_v05_guid_token=news_v05[["guid","title_and_body_token"]]
# news_v05_guid_token.to_pickle("D:4.AutoencoderForArticle.Mid-Industry/news_v05_guid_token.p")

#### build tokenizer on all de-duplicate single industry News

In [28]:
from keras.preprocessing.text import Tokenizer

C:\Users\garyyu\AppData\Local\Continuum\anaconda3\envs\deeplearningcpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [29]:
tokenizer=Tokenizer()

In [30]:
tokenizer.fit_on_texts(text)

In [33]:
print("Total words counts:",len(tokenizer.word_counts))

Total words counts: 506962


In [34]:
word_counts=pd.Series(tokenizer.word_counts)
word_counts=word_counts.sort_values(ascending=False)

In [43]:
word_counts

億元      601312
營收      584488
今年      540844
市場      535955
表示      521515
公司      480517
美元      402166
成長      394434
產品      346322
指出      345462
目前      328650
預估      306569
美國      303536
全球      284853
去年      281562
預期      276678
需求      259513
中國      237827
大陸      223549
獲利      222859
持續      218453
指數      215201
報導      213817
日圓      211238
以及      210925
年增      210810
價格      210646
增加      207366
影響      204050
客戶      201726
         ...  
造園           1
蘇點忠          1
甘錫瀅          1
李華武          1
riva         1
拈花惹草         1
進八強          1
齊擬配          1
光過           1
啟後台          1
創作技巧         1
卡可分          1
廣華配          1
加包仿          1
廣華繼          1
旅是           1
提述           1
杯熱           1
稱賽           1
蹦解           1
那標           1
將塞普          1
取大者          1
人不選          1
宮崎駿          1
杜喜           1
荷比法          1
港圳           1
枕及           1
金紗           1
Length: 506962, dtype: int64

In [39]:
with open("D:4.AutoencoderForArticle.Mid-Industry/tokenizer.v05.p","wb") as f:
    pickle.dump(tokenizer,f)

#### build tokenizer for individual industry

In [77]:
News_1=news_v05[["guid","title_and_body_token"]]
News_2=newswithindustry_v04[["guid","medi-industry"]]
News_1["guid"]=News_1["guid"].apply(lambda x : uuid.UUID(x) )
News=pd.merge(News_1,News_2,how="inner",left_on="guid",right_on="guid")
News["medi-industry"]=News["medi-industry"].apply(lambda x : list(x)[0] if len(list(x))==1 else None )

C:\Users\garyyu\AppData\Local\Continuum\anaconda3\envs\deeplearningcpu\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [90]:
tokenizers={}
for i in tqdm_notebook(industry):
    tokenizer_partial=Tokenizer()
    text_partial=News[News["medi-industry"]==i]["title_and_body_token"]
    tokenizer_partial.fit_on_texts(text_partial)
    tokenizers[i]=tokenizer_partial
    

In [93]:
with open("D:4.AutoencoderForArticle.Mid-Industry/tokenizers.v05.p","wb") as f:
    pickle.dump(tokenizers,f)